# Core Tutorial

This script will introduce you to the basics of time series handling with pynapple.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pynapple as nap

## Time series object

Let's create a Tsd object with fake data. In this case, every time point is 1 second apart. A Tsd object is a wrapper of pandas series.

In [2]:
tsd = nap.Tsd(t = np.arange(100), d = np.random.rand(100), time_units = 's')

print(tsd)

Time (s)
0.0     0.671375
1.0     0.184115
2.0     0.745202
3.0     0.041667
4.0     0.844270
          ...   
95.0    0.224841
96.0    0.011205
97.0    0.904223
98.0    0.330948
99.0    0.288449
Length: 100, dtype: float64


While the tsd object appears in second, it actually holds the values in microseconds by default. It is possible to switch between seconds, milliseconds and microseconds. Note that when using *as_units*, the returned object is a simple pandas series.

In [3]:
print(tsd.as_units('ms'))
print(tsd.as_units('us'))

Time (ms)
0.0        0.671375
1000.0     0.184115
2000.0     0.745202
3000.0     0.041667
4000.0     0.844270
             ...   
95000.0    0.224841
96000.0    0.011205
97000.0    0.904223
98000.0    0.330948
99000.0    0.288449
Length: 100, dtype: float64
Time (us)
0           0.671375
1000000     0.184115
2000000     0.745202
3000000     0.041667
4000000     0.844270
              ...   
95000000    0.224841
96000000    0.011205
97000000    0.904223
98000000    0.330948
99000000    0.288449
Length: 100, dtype: float64


If only timestamps are available, for example spike times, we can construct a Ts object which holds only times. In this case, we generate 10 random spike times between 0 and 100 ms.

In [4]:
ts = nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 'ms')

print(ts)

Time (s)
0.011981   NaN
0.018039   NaN
0.021218   NaN
0.042355   NaN
0.059233   NaN
0.062366   NaN
0.069332   NaN
0.073728   NaN
0.091412   NaN
0.093179   NaN
dtype: float64


If the time series contains multiple columns, we can use a TsdFrame.

In [5]:
tsdframe = nap.TsdFrame(t = np.arange(100), d = np.random.rand(100,3), time_units = 's', columns = ['a', 'b', 'c'])

print(tsdframe)

                 a         b         c
Time (s)                              
0.0       0.070958  0.248339  0.678089
1.0       0.807558  0.782743  0.895054
2.0       0.659416  0.937503  0.516411
3.0       0.852223  0.219005  0.194536
4.0       0.170820  0.639453  0.818144
...            ...       ...       ...
95.0      0.565494  0.437288  0.370130
96.0      0.904865  0.298250  0.590778
97.0      0.697050  0.150686  0.462217
98.0      0.040829  0.311084  0.668681
99.0      0.150205  0.279965  0.883045

[100 rows x 3 columns]


## Interval Sets object

The [IntervalSet](https://peyrachelab.github.io/pynapple/core.interval_set/) object stores multiple epochs with a common time units. It can then be used to restrict time series to this particular set of epochs.

In [6]:
epochs = nap.IntervalSet(start = [0, 10], end = [5, 15], time_units = 's')

new_tsd = tsd.restrict(epochs)

print(epochs)
print('\n')
print(new_tsd)

   start   end
0    0.0   5.0
1   10.0  15.0


Time (s)
1.0     0.184115
2.0     0.745202
3.0     0.041667
4.0     0.844270
5.0     0.670595
11.0    0.844376
12.0    0.920450
13.0    0.042605
14.0    0.662674
15.0    0.001379
dtype: float64


Multiple operations are available for IntervalSet. For example, IntervalSet can be merged. See the full documentation of the class [here](https://peyrachelab.github.io/pynapple/core.interval_set/#pynapple.core.interval_set.IntervalSet.intersect) for a list of all the functions that can be used to manipulate IntervalSets.

In [7]:
epoch1 = nap.IntervalSet(start=[0], end=[10]) # no time units passed. Default is us.
epoch2 = nap.IntervalSet(start=[5,30],end=[20,45])

epoch = epoch1.union(epoch2)
print(epoch1, '\n')
print(epoch2, '\n')
print(epoch)

   start      end
0    0.0  0.00001 

      start       end
0  0.000005  0.000020
1  0.000030  0.000045 

     start       end
0  0.00000  0.000020
1  0.00003  0.000045


## TsGroup

Multiple time series with different time stamps (.i.e. a group of neurons with different spike times from one session) can be grouped with the TsGroup object. The TsGroup behaves like a dictionnary but it is also possible to slice with a list of indexes

In [8]:
my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 1000)), time_units = 's'), # here a simple dictionnary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 2000)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 3000)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

print(tsgroup, '\n')
print(tsgroup[0], '\n') # dictionnary like indexing returns directly the Ts object
print(tsgroup[[0,2]]) # list like indexing

  Index    Freq. (Hz)
-------  ------------
      0         10.02
      1         19.98
      2         29.98 

Time (s)
0.432052    NaN
0.529947    NaN
0.601565    NaN
0.774862    NaN
0.799916    NaN
             ..
99.579318   NaN
99.647440   NaN
99.680493   NaN
99.836327   NaN
99.848397   NaN
Length: 999, dtype: float64 

  Index    Freq. (Hz)
-------  ------------
      0         10.02
      2         29.98


Operations such as restrict can thus be directly applied to the TsGroup as well as other operations.

In [9]:
newtsgroup = tsgroup.restrict(epochs)

count = tsgroup.count(1, epochs, time_units='s') # Here counting the elements within bins of 1 seconds

print(count)

           0   1   2
Time (s)            
0.5        6  13  31
1.5        8  25  29
2.5        7  26  22
3.5       13  16  32
4.5       14  20  32
10.5       8  22  29
11.5      10  21  27
12.5       6  26  29
13.5       9  18  45
14.5      13  19  26


One advantage of grouping time series is that metainformation can be added about each elements. In this case, we add labels to each Ts object when instantiating the group and after. We can then use this label to split the group. See the documentation about [TsGroup](https://peyrachelab.github.io/pynapple/core.ts_group/) for all the ways to split TsGroup.

In [10]:
tsgroup = nap.TsGroup(my_ts, time_units = 's', label1=[0,1,0])
tsgroup.set_info(label2=['a', 'a', 'b'])

print(tsgroup, '\n')

newtsgroup= tsgroup.getby_category('label1')
print(newtsgroup[0], '\n')
print(newtsgroup[1])


  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0         10.02         0  a
      1         19.98         1  a
      2         29.98         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0         10.02         0  a
      2         29.98         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      1         19.98         1  a


## Time support

A key element of the manipulation of time series by pynapple is the inherent time support defined for Ts, Tsd, TsdFrame and TsGroup objects. The time support is defined as an IntervalSet that provides the time serie with a context. For example,, the restrict operation will update automatically the time support to the new time series. Ideally the time support should be defined for all time series when instantiating them. If no time series is given, the time support is inferred from the start and end of the time series. 

In this example, a TsGroup is instantiated with and without a time support. Notice how the frequency of each Ts element is changed when the time support is defined explicitely.

In [11]:
time_support = nap.IntervalSet(start = 0, end = 100, time_units = 's')

my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 's'), # here a simple dictionnary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 20)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 30)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

tsgroup_with_time_support = nap.TsGroup(my_ts, time_support = time_support)

print(tsgroup, '\n')

print(tsgroup_with_time_support, '\n')

print(tsgroup_with_time_support.time_support) # acceding the time support

  Index    Freq. (Hz)
-------  ------------
      0          0.14
      1          0.16
      2          0.36 

  Index    Freq. (Hz)
-------  ------------
      0           0.1
      1           0.2
      2           0.3 

   start    end
0    0.0  100.0
